In [9]:
%run /OEA_py

StatementMeta(spark3p0smallc, 6, -1, Finished, Available)

In [2]:
%run /Clever_py

StatementMeta(spark3p0smallc, 6, -1, Finished, Available)

In [3]:
%run /ContosoSIS_py

StatementMeta(spark3p0smallc, 6, -1, Finished, Available)

In [4]:
%run /iReady_py

StatementMeta(spark3p0smallc, 6, -1, Finished, Available)

In [5]:
%run /M365_py

StatementMeta(spark3p0smallc, 6, -1, Finished, Available)

In [10]:
oea = OEA('stoeacisd3ggimpl1', '2fc72a29-e1ee-4eb6-95eb-9346557a6d6d', 'salt123')
m365 = M365(oea)
clever = Clever(oea)
contoso_sis = ContosoSIS(oea)
iready = IReady(oea)

# Copy the test data sets into stage1 (simulating the process of landing data into stage1 of your data lake).
#clever.copy_test_data_to_stage1()
#contoso_sis.copy_test_data_to_stage1()
#iready.copy_test_data_to_stage1()
#m365.copy_test_data_to_stage1()

StatementMeta(spark3p0smallc, 6, 10, Finished, Available)

2021-08-19 20:33:15,716 - __main__ - DEBUG - OEA initialized.

In [221]:
m365.process_roster()

In [12]:
#clever.process_data_from_stage1()
clever.create_stage2_db('PARQUET')

#contoso_sis.process_data_from_stage1()
contoso_sis.create_stage2_db('PARQUET')

#iready.process_data_from_stage1()
iready.create_stage2_db('PARQUET')

StatementMeta(spark3p0smallc, 6, 12, Finished, Available)

In [24]:
m365.process_activity()
m365.create_stage2_db('PARQUET')

StatementMeta(spark3p0smallc, 6, 24, Finished, Available)

In [171]:
m365.reset_activity_processing()
m365.reset_roster_processing()

# reset everything

In [281]:
contoso_sis.delete_all_stages()
clever.delete_all_stages()
iready.delete_all_stages()

In [17]:
sqlContext.registerDataFrameAsTable(spark.read.format('parquet').load(oea.stage2p + '/contoso_sis/studentsectionmark'), 'SectionMark')

sqlContext.registerDataFrameAsTable(spark.read.format('parquet').load(oea.stage2np + '/m365/Person_lookup'), 'Person_lookup')
sqlContext.registerDataFrameAsTable(spark.read.format('parquet').load(oea.stage2p + '/m365/Section'), 'Section')

df = spark.sql("select sm.id Id, pl.Id PersonId, s.Id SectionId, cast(sm.numeric_grade_earned as int) NumericGrade, \
sm.alpha_grade_earned AlphaGrade, sm.is_final_grade IsFinalGrade, cast(sm.credits_attempted as int) CreditsAttempted, cast(sm.credits_earned as int) CreditsEarned, \
sm.grad_credit_type GraduationCreditType, sm.id ExternalId, CURRENT_TIMESTAMP CreateDate, CURRENT_TIMESTAMP LastModifiedDate, true IsActive \
from SectionMark sm, Person_lookup pl, Section s \
where sm.student_id_pseudonym = pl.ExternalId_pseudonym \
and sm.section_id = s.ExternalId")

display(df)

StatementMeta(spark3p0smallc, 6, 17, Finished, Available)

SynapseWidget(Synapse.DataFrame, 6c9110d0-79be-4240-877d-1c6366dcfa55)

In [19]:
sqlContext.registerDataFrameAsTable(spark.read.format('parquet').load(oea.stage2p + '/contoso_sis/studentsectionmark'), 'SectionMark')

sqlContext.registerDataFrameAsTable(spark.read.format('parquet').load(oea.stage2np + '/m365/Person_lookup'), 'Person_lookup')
sqlContext.registerDataFrameAsTable(spark.read.format('parquet').load(oea.stage2p + '/m365/Section'), 'Section')

df = spark.sql("select sm.id Id, pl.Id PersonId, s.Id SectionId, cast(sm.numeric_grade_earned as int) NumericGrade, \
sm.alpha_grade_earned AlphaGrade, sm.is_final_grade IsFinalGrade, cast(sm.credits_attempted as int) CreditsAttempted, cast(sm.credits_earned as int) CreditsEarned, \
sm.grad_credit_type GraduationCreditType, sm.id ExternalId, CURRENT_TIMESTAMP CreateDate, CURRENT_TIMESTAMP LastModifiedDate, true IsActive \
from SectionMark sm, Person_lookup pl, Section s \
where sm.student_id_pseudonym = pl.ExternalId_pseudonym \
and sm.section_id = s.ExternalId")

df.write.format('parquet').mode('overwrite').save(oea.stage2np + '/ContosoISD/SectionMark')
df.write.format('parquet').mode('overwrite').save(oea.stage2np + '/ContosoISD/SectionMark2')

# Repeat the above process, this time for student attendance
# Convert id values to use the Person.Id, Org.Id and Section.Id values
sqlContext.registerDataFrameAsTable(spark.read.format('parquet').load(oea.stage2p + '/contoso_sis/studentattendance'), 'Attendance')
sqlContext.registerDataFrameAsTable(spark.read.format('parquet').load(oea.stage2p + '/m365/Org'), 'Org')

df = spark.sql("select att.id Id, pl.Id PersonId, att.school_year SchoolYear, o.Id OrgId, to_date(att.attendance_date,'MM/dd/yyyy') AttendanceDate, \
att.all_day AllDay, att.Period Period, s.Id SectionId, att.AttendanceCode AttendanceCode, att.PresenceFlag PresenceFlag, \
att.attendance_status AttendanceStatus, att.attendance_type AttendanceType, att.attendance_sequence AttendanceSequence \
from Attendance att, Org o, Person_lookup pl, Section s \
where att.student_id_pseudonym = pl.ExternalId_pseudonym \
and att.school_id = o.ExternalId \
and att.section_id = s.ExternalId")

display(df)

StatementMeta(spark3p0smallc, 6, 19, Finished, Available)

SynapseWidget(Synapse.DataFrame, 0b625f45-c84b-47c0-95e9-6887584f8567)

In [20]:
# Extracted from Contoso_ISD_setup_and_update

# Process sectionmark data
# Convert id values to use the Person.Id and Section.Id values set in the Education Data Platform.

sqlContext.registerDataFrameAsTable(spark.read.format('parquet').load(oea.stage2p + '/contoso_sis/studentsectionmark'), 'SectionMark')

sqlContext.registerDataFrameAsTable(spark.read.format('parquet').load(oea.stage2np + '/m365/Person_lookup'), 'Person_lookup')
sqlContext.registerDataFrameAsTable(spark.read.format('parquet').load(oea.stage2p + '/m365/Section'), 'Section')

df = spark.sql("select sm.id Id, pl.Id PersonId, s.Id SectionId, cast(sm.numeric_grade_earned as int) NumericGrade, \
sm.alpha_grade_earned AlphaGrade, sm.is_final_grade IsFinalGrade, cast(sm.credits_attempted as int) CreditsAttempted, cast(sm.credits_earned as int) CreditsEarned, \
sm.grad_credit_type GraduationCreditType, sm.id ExternalId, CURRENT_TIMESTAMP CreateDate, CURRENT_TIMESTAMP LastModifiedDate, true IsActive \
from SectionMark sm, Person_lookup pl, Section s \
where sm.student_id_pseudonym = pl.ExternalId_pseudonym \
and sm.section_id = s.ExternalId")

df.write.format('parquet').mode('overwrite').save(oea.stage2np + '/ContosoISD/SectionMark')
df.write.format('parquet').mode('overwrite').save(oea.stage2np + '/ContosoISD/SectionMark2')

# Repeat the above process, this time for student attendance
# Convert id values to use the Person.Id, Org.Id and Section.Id values
sqlContext.registerDataFrameAsTable(spark.read.format('parquet').load(oea.stage2p + '/contoso_sis/studentattendance'), 'Attendance')
sqlContext.registerDataFrameAsTable(spark.read.format('parquet').load(oea.stage2p + '/m365/Org'), 'Org')

df = spark.sql("select att.id Id, pl.Id PersonId, att.school_year SchoolYear, o.Id OrgId, to_date(att.attendance_date,'MM/dd/yyyy') AttendanceDate, \
att.all_day AllDay, att.Period Period, s.Id SectionId, att.AttendanceCode AttendanceCode, att.PresenceFlag PresenceFlag, \
att.attendance_status AttendanceStatus, att.attendance_type AttendanceType, att.attendance_sequence AttendanceSequence \
from Attendance att, Org o, Person_lookup pl, Section s \
where att.student_id_pseudonym = pl.ExternalId_pseudonym \
and att.school_id = o.ExternalId \
and att.section_id = s.ExternalId")

df.write.format('parquet').mode('overwrite').save(oea.stage2np +'/ContosoISD/Attendance')


# Add 'Department' column to Course (hardcoded to "Math" for this Contoso example)
sqlContext.registerDataFrameAsTable(spark.read.format('parquet').load(oea.stage2p + '/m365/Course'), 'Course')
df = spark.sql("select Id, Name, Code, Description, ExternalId, CreateDate, LastModifiedDate, IsActive, CalendarId, 'Math' Department from Course")
df.write.format('parquet').mode('overwrite').save(oea.stage2np + '/ContosoISD/Course')




StatementMeta(spark3p0smallc, 6, 20, Finished, Available)

In [25]:
# Create spark db to allow for access to the data in the delta-lake via SQL on-demand.
# This is only creating metadata for SQL on-demand, pointing to the data in the delta-lake.
# This also makes it possible to connect in Power BI via the azure sql data source connector.
db_name = 's2_ContosoISD'
spark.sql('CREATE DATABASE IF NOT EXISTS ' + db_name)
spark.sql(f"create table if not exists " + db_name + ".Activity using PARQUET location '" + oea.stage2p + "/m365/TechActivity'")
spark.sql(f"create table if not exists " + db_name + ".Calendar using PARQUET location '" + oea.stage2p + "/m365/Calendar'")
spark.sql(f"create table if not exists " + db_name + ".Org using PARQUET location '" + oea.stage2p + "/m365/Org'")
spark.sql(f"create table if not exists " + db_name + ".Person using PARQUET location '" + oea.stage2p + "/m365/Person'")
spark.sql(f"create table if not exists " + db_name + ".PersonIdentifier using PARQUET location '" + oea.stage2p + "/m365/PersonIdentifier'")
spark.sql(f"create table if not exists " + db_name + ".RefDefinition using PARQUET location '" + oea.stage2p + "/m365/RefDefinition'")
spark.sql(f"create table if not exists " + db_name + ".Section using PARQUET location '" + oea.stage2p + "/m365/Section'")
spark.sql(f"create table if not exists " + db_name + ".Session using PARQUET location '" + oea.stage2p + "/m365/Session'")
spark.sql(f"create table if not exists " + db_name + ".StaffOrgAffiliation using PARQUET location '" + oea.stage2p + "/m365/StaffOrgAffiliation'")
spark.sql(f"create table if not exists " + db_name + ".StaffSectionMembership using PARQUET location '" + oea.stage2p + "/m365/StaffSectionMembership'")
spark.sql(f"create table if not exists " + db_name + ".StudentOrgAffiliation using PARQUET location '" + oea.stage2p + "/m365/StudentOrgAffiliation'")
spark.sql(f"create table if not exists " + db_name + ".StudentSectionMembership using PARQUET location '" + oea.stage2p + "/m365/StudentSectionMembership'")

spark.sql(f"create table if not exists " + db_name + ".Course using PARQUET location '" + oea.stage2np + "/ContosoISD/Course'")
spark.sql(f"create table if not exists " + db_name + ".Attendance using PARQUET location '" + oea.stage2np + "/ContosoISD/Attendance'")
spark.sql(f"create table if not exists " + db_name + ".SectionMark using PARQUET location '" + oea.stage2np + "/ContosoISD/SectionMark'")
spark.sql(f"create table if not exists " + db_name + ".SectionMark2 using PARQUET location '" + oea.stage2np + "/ContosoISD/SectionMark2'")


StatementMeta(spark3p0smallc, 6, 25, Finished, Available)

DataFrame[]